In [370]:
import numpy as np
from typing import List, Tuple
from math import gcd
import random
import logging

def egcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = egcd(b % a, a)
        return g, x - (b // a) * y, y


def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [365]:

# Sieve of Eratosthenes
# Code by David Eppstein, UC Irvine, 28 Feb 2002
# http://code.activestate.com/recipes/117119/

def primes_generator():
    """ Generate an infinite sequence of prime numbers.
    """
    # Maps composites to primes witnessing their compositeness.
    # This is memory efficient, as the sieve is not "run forward"
    # indefinitely, but only as long as required by the current
    # number being tested.
    #
    D = {}
    
    # The running integer that's checked for primeness
    q = 2
    
    while True:
        if q not in D:
            # q is a new prime.
            # Yield it and mark its first multiple that isn't
            # already marked in previous iterations
            # 
            yield q
            D[q * q] = [q]
        else:
            # q is composite. D[q] is the list of primes that
            # divide it. Since we've reached q, we no longer
            # need it in the map, but we'll mark the next 
            # multiples of its witnesses to prepare for larger
            # numbers
            # 
            for p in D[q]:
                D.setdefault(p + q, []).append(p)
            del D[q]
        
        q += 1

In [366]:
def generate_primes(n: int):
    primes = []
    i = 0
    for p in primes_generator():
        i += 1
        primes.append(p)
        if i == n:
            return primes

In [222]:
BIG_PRIMES_CACHE_10k = generate_primes(100000)[90000:]

In [223]:
BIG_PRIMES_CACHE_10k[0]

1159531

## Задание 1

Реализовать алгоритмы генерации ключей шифровки и дешифровки. Протестировать их для шифрования и дешифрования произвольных сообщений.

In [367]:
def prime_mod_inv(x, mod):
    return pow(x, mod - 2, mod)

In [371]:
def generate_keys(primes_db: List[int] = BIG_PRIMES_CACHE_10k) -> Tuple[Tuple[int], Tuple[int]]:
    p, q = np.random.choice(primes_db, size = 2, replace = False)
    p, q = int(p), int(q)
    n = q*p
    print( f"Primes selected: p={p}, q={q}.\np*q = {n}")
    
    phi = (p-1)*(q-1)
    print( f"phi = {phi}")
    
    while True:
        e = int (random.randint(1, n) )
        if gcd(e, phi) == 1:
            break
    print( f"e = {e}")
    
    d = modinv(e, phi)
    print((d*e)%phi)
    assert (d*e)%phi == 1
    print( f"d = {phi}")
    
    public_key = (e, n)
    priate_key = (d, n)

    return public_key, priate_key

In [372]:
generate_keys()

Primes selected: p=1178669, q=1291219.
p*q = 1521919807511
phi = 1521917337624
e = 370810455565
1
d = 1521917337624


((370810455565, 1521919807511), (64108792285, 1521919807511))

## Проверим

In [373]:
import time

In [387]:
def encrypt(message: List[int], pub_key: Tuple[int]) -> List[int]:
    e, n = pub_key
    return [pow(m, e, n) for m in message]
     
def decrypt(code: List[int], private_key: Tuple[int]) -> List[int]:
    d, n=private_key
    return [pow(c, d, n) for c in code]

In [388]:
pub_key, private_key = generate_keys()
message = [55, 66, 89, 78, 93, 22]
code = encrypt(message, pub_key)
mesage_restored = decrypt(code, private_key)

message == mesage_restored

Primes selected: p=1194883, q=1252103.
p*q = 1496116588949
phi = 1496114141964
e = 555701013949
1
d = 1496114141964


True

In [389]:
pub_key

(555701013949, 1496116588949)

In [390]:
message

[55, 66, 89, 78, 93, 22]

In [391]:
mesage_restored

[55, 66, 89, 78, 93, 22]

## Задание 2

Вариант 5

### a) 

Пусть даны открытые ключи {n,e} и сообщение m. Зашифруйте используя криптосистему RSA.

In [392]:
m = 3678
e = 176561525
n = 279976601

In [393]:
pub_key = (e, n)

code = encrypt([m], pub_key)

code

[138387333]

## b)

Расшифровать полученные шифротексты используя криптосистему RSA. Сравните результаты с исходным текстом.

In [394]:
cp = 15 / 3
cp * 3 == 15

True

In [395]:
def factorize_into_primes(num: int) -> List[int]:
    """ Assume that num is a combination of exactly 2 primes"""
    for prime in primes_generator():
        co_prime = num // prime
        if co_prime * prime == num:
            return [co_prime , prime]

In [396]:
p, q = factorize_into_primes(n)

phi = (p - 1) * (q - 1)

d = modinv(e, phi)

p, q, d

(17623, 15887, 269809349)

In [397]:
private_key = (d, n)

In [398]:
decrypt(code, private_key)

[3678]

In [399]:
m

3678